In [18]:
%run ./Preprocessing.ipynb

import optuna
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import (
    StratifiedKFold, TimeSeriesSplit, KFold
)
from sklearn.model_selection import cross_val_score


Preprocessed : xtrain, xtest, ytrain, ytest
(74736, 13) (18684, 13) (74736,) (18684,)


In [19]:
lr = LogisticRegression()

In [20]:
opt_grid = {
            'cv': KFold(n_splits=10, random_state=None, shuffle=True),
            'number_of_trials': 100,
            'time_constraint': 60 * 1
        }

static_param = {
            "penalty": 'elasticnet',
             'dual': False,
             'solver':'saga'
        }

In [22]:
def objective(trial):
    '''
    Objective function to optimise the hyperparamers of Logistic Regression 
    Args:
         trial object (object): Trials evaluating an objective function. 
         This object provides interfaces to get parameter suggestion, manage the trial’s state, and set/get user-defined attributes of the trial.

    Returns:
          Study  (object): Trained Study Object
    '''  
    
    param_grid = {
                "intercept_scaling": trial.suggest_float("intercept_scaling", 0.1, 1.0),
                "max_iter": trial.suggest_float("max_iter", 300, 300),
                "l1_ratio": trial.suggest_float("l1_ratio", 0.001, 0.9999)
            }

    full_grid = {**param_grid, **static_param}

    cv = cross_val_score(estimator=lr,
                                X=xtrain,
                                y=ytrain,
                                cv= opt_grid['cv'],
                                scoring='roc_auc',
                                verbose=True)

    score = cv.mean()

    if trial.should_prune():
                raise optuna.exceptions.TrialPruned()

    return score
    
optuna.logging.set_verbosity(optuna.logging.ERROR)
study = optuna.create_study(sampler=optuna.samplers.TPESampler(), direction='maximize')
study.optimize(objective, n_trials=opt_grid['number_of_trials'], show_progress_bar=False, timeout=opt_grid['time_constraint'])
model_parameters = study.best_params

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    2.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    2.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    3.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    2.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    2.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    2.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_j

In [23]:
#Instatiate a Logistic Regression Model 
lr_model = LogisticRegression()

In [24]:
#Set the parameters to the tuned parameters from the optuna study
lr_model.set_params(**model_parameters, **static_param)

LogisticRegression(intercept_scaling=0.6855164633757622,
                   l1_ratio=0.3687970636941655, max_iter=300.0,
                   penalty='elasticnet', solver='saga')

In [25]:
#Fit the model on the training data
model_lr=lr_model.fit(xtrain,ytrain)
#Make predictions. 
pred_lr=model_lr.predict(xtest)
probs_lr = model_lr.predict_proba(xtest)[:, 1]

pred_lr_train = model_xgb.predict(xtrain)
probs_lr_train =  model_xgb.predict_proba(xtrain)[:, 1]

print('-----------model_lr, pred_lr, probs_lr loaded-----------')

c:\dev\.conda\envs\jupyter-lab\lib\site-packages\sklearn\linear_model\_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


NameError: name 'model_xgb' is not defined